In [4]:
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [2]:
df_dev  = pd.read_csv("../dados/ready/dev.csv")
df_comp = pd.read_csv("../dados/ready/comp.csv")

In [3]:
def get_x(df: pd.DataFrame) -> None:
    condition = (df.columns != 'status') & (df.columns != 'account_id') & (df.columns != 'loan_id') & (
        df.columns != 'client_id') & (df.columns != 'district_id') & (df.columns != 'disp_id') & (df.columns != 'district_id_y') & (
        df.columns != 'district_id_x') & (df.columns != 'id')
    return df.loc[:, condition]

def get_y(df: pd.DataFrame) -> None:
    return df.status

In [10]:
start = time.time()

train, test = train_test_split(df_dev, test_size = 0.2)

x = get_x(train)
y = get_y(train)

lr_param_grid = { 
    'C' : [0.1, 1, 10, 100], 
    'penalty' : ['l1', 'l2', 'elasticnet'],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
log_reg = GridSearchCV(LogisticRegression(), param_grid=lr_param_grid, cv= 5)

log_reg.fit(x, y)

end = time.time()

pred_competition = log_reg.predict_proba(get_x(df_comp))

prediction = pd.DataFrame({'Id': df_comp['loan_id'], 'Predicted': pred_competition[::, 1]})

predicted = log_reg.predict_proba(get_x(test))[::, 1]
expected  = get_y(test)
score = roc_auc_score(expected, predicted)

print(f"Score is {score}")

print(f"Time Elapsed: {end-start}")

C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\mika1\AppData\R

Score is 0.9610136452241715
Time Elapsed: 7.176156282424927


C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\mika1\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\mika1\AppDa

In [11]:
prediction.to_csv("../predictions/logistic-grid.csv", index = False)